In [1]:
import sqlalchemy as sa
import pandas as pd

from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

from matplotlib.animation import FuncAnimation, writers
import numpy as np

In [32]:
SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

team_id = 4377
division_id = 1018

In [73]:
team_df = pd.read_sql_table(table_name='Team', con=engine)
team_df = team_df[team_df['TeamId'] == team_id]
team_members_df = pd.read_sql_table(table_name='TeamMember', con=engine)
players_df = pd.read_sql_table(table_name='Player', con=engine)
team_members_df = pd.merge(players_df, team_members_df, left_on='PlayerId', right_on='PlayerId')
df = pd.merge(team_df, team_members_df, left_on='TeamId', right_on='TeamId')
map_df = pd.read_sql_table(table_name='Map', con=engine)
hero_df = pd.read_sql_table(table_name='Hero', con=engine)
hero_df = hero_df[['HeroName', 'NewRole', 'HeroId']]

In [75]:
print(f'{df.shape[0]}')
df.head()

6


,TeamId,TeamName,TeamNameLower,Ticker,TickerLower,PlayerId,ShortBattletag,Battletag,AccountLevel,BattletagUriEscape,...,TeamMemberId,TeamLeagueDivisionId,IsCaptain,IsAssistantCaptain,IsSmurf,LeagueProfileUrl,IsActive,DateJoined,DateLeft,DivisionId
0,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11589,Javedesi,Javedesi#1399,425,Javedesi%231399,...,317,959.0,False,False,False,https://www.nexusgamingseries.org/profile/Jave...,True,NaT,NaT,1018.0
1,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11591,cactipus,cactipus#1355,257,cactipus%231355,...,319,959.0,False,False,False,https://www.nexusgamingseries.org/profile/cact...,True,NaT,NaT,1018.0
2,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11593,OmarG,OmarG#1399,564,OmarG%231399,...,321,959.0,False,False,False,https://www.nexusgamingseries.org/profile/Omar...,True,NaT,NaT,1018.0
3,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11597,AlixGx,AlixGx#1722,200,AlixGx%231722,...,325,959.0,False,False,False,https://www.nexusgamingseries.org/profile/Alix...,True,NaT,NaT,1018.0
4,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11598,SmackLoud,SmackLoud#1757,391,SmackLoud%231757,...,326,959.0,False,False,False,https://www.nexusgamingseries.org/profile/Smac...,True,NaT,NaT,1018.0


In [76]:
replays = pd.read_sql_table(table_name='Replay', con=engine)
team_member_replays_df = pd.merge(df, replays, left_on='Battletag', right_on='Battletag')
team_member_replays_df = pd.merge(team_member_replays_df, hero_df, left_on='Hero', right_on='HeroName')

,TeamId,TeamName,TeamNameLower,Ticker,TickerLower,PlayerId,ShortBattletag,Battletag,AccountLevel,BattletagUriEscape,...,RegenGlobes,Region_y,PlayerMmr,HeroMmr,RoleMmr,MmrDateParsed,GameVersion,HeroName,NewRole,HeroId
0,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11589,Javedesi,Javedesi#1399,425,Javedesi%231399,...,0,1,1685.21,1760.54,1717.87,2022-01-14 06:32:08,None,Artanis,Bruiser,7
1,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11589,Javedesi,Javedesi#1399,425,Javedesi%231399,...,12,1,2589.70,1742.60,2282.73,2022-03-09 14:54:20,None,Artanis,Bruiser,7
2,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11589,Javedesi,Javedesi#1399,425,Javedesi%231399,...,27,1,1897.61,1919.51,1917.26,2022-03-09 14:42:59,None,Artanis,Bruiser,7
3,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11591,cactipus,cactipus#1355,257,cactipus%231355,...,11,1,1736.13,1727.55,1725.98,2021-04-24 02:16:08,None,Artanis,Bruiser,7
4,4377,Hard Stuck Support Group,hard stuck support group,hssg,hssg,11591,cactipus,cactipus#1355,257,cactipus%231355,...,22,1,2101.24,1828.48,1817.34,2021-05-17 23:46:03,None,Artanis,Bruiser,7


In [77]:
print(f'Total Replays for all members: {team_member_replays_df.shape[0]}')
player_totals = team_member_replays_df[['Battletag', 'AccountLevel', 'SourceReplayId', 'IsWinner', 'Hero', 'NewRole', 'GameMap', 'GameType', 'GameDate', 'Team']]
team_replays = player_totals[['SourceReplayId', 'Team', 'Battletag', 'Hero']].groupby(['SourceReplayId', 'Team']).size().reset_index(name='TotalGames').sort_values(by='TotalGames', ascending=False)
team_replays = team_replays[team_replays['TotalGames'] == 5]
r_df = pd.merge(player_totals, team_replays, left_on=['SourceReplayId', 'Team'], right_on=['SourceReplayId', 'Team'])
print(f'{r_df.shape[0]}')
r_df.head(1000)

Total Replays for all members: 6208
675


,Battletag,AccountLevel,SourceReplayId,IsWinner,Hero,NewRole,GameMap,GameType,GameDate,Team,TotalGames
0,cactipus#1355,257,42074032,1,Artanis,Bruiser,Infernal Shrines,Quick Match,2022-02-01 04:00:41,0,5
1,SmackLoud#1757,391,42074032,1,Li Li,Healer,Infernal Shrines,Quick Match,2022-02-01 04:00:41,0,5
2,Javedesi#1399,425,42074032,1,Li-Ming,Ranged Assassin,Infernal Shrines,Quick Match,2022-02-01 04:00:41,0,5
3,OmarG#1399,564,42074032,1,Hanzo,Ranged Assassin,Infernal Shrines,Quick Match,2022-02-01 04:00:41,0,5
4,RndmEngineer#1403,1069,42074032,1,Blaze,Tank,Infernal Shrines,Quick Match,2022-02-01 04:00:41,0,5
...,...,...,...,...,...,...,...,...,...,...,...
670,OmarG#1399,564,42148718,0,Hanzo,Ranged Assassin,Braxis Holdout,Storm League,2022-02-04 04:02:48,0,5
671,Javedesi#1399,425,42148718,0,Lt. Morales,Healer,Braxis Holdout,Storm League,2022-02-04 04:02:48,0,5
672,SmackLoud#1757,391,42148718,0,Azmodan,Ranged Assassin,Braxis Holdout,Storm League,2022-02-04 04:02:48,0,5
673,cactipus#1355,257,42148718,0,Rexxar,Bruiser,Braxis Holdout,Storm League,2022-02-04 04:02:48,0,5


In [81]:
player_games = r_df.groupby(['Battletag']).size().reset_index(name='TotalGames').sort_values(by='TotalGames', ascending=False)
player_games.head(100)

,Battletag,TotalGames
4,SmackLoud#1757,132
3,RndmEngineer#1403,131
2,OmarG#1399,130
5,cactipus#1355,120
1,Javedesi#1399,115
0,AlixGx#1722,47


In [89]:
player_heroes = r_df.groupby(['Hero', 'Battletag']).size().reset_index(name='TotalGames').sort_values(by='TotalGames', ascending=False)
player_heroes.head(100)

,Hero,Battletag,TotalGames
104,Sonya,Javedesi#1399,31
10,Artanis,cactipus#1355,30
46,Hanzo,OmarG#1399,26
85,Mei,RndmEngineer#1403,26
33,E.T.C.,RndmEngineer#1403,20
...,...,...,...
131,Varian,cactipus#1355,1
135,Yrel,AlixGx#1722,1
117,Sylvanas,SmackLoud#1757,1
127,Uther,cactipus#1355,1


In [102]:
player_maps = r_df.groupby(['GameMap']).size().reset_index(name='TotalGames').sort_values(by='TotalGames', ascending=False)
player_maps.head(100)

,GameMap,TotalGames
1,Battlefield of Eternity,120
12,Towers of Doom,80
9,Infernal Shrines,65
0,Alterac Pass,60
11,Tomb of the Spider Queen,55
3,Braxis Holdout,50
4,Cursed Hollow,40
5,Dragon Shire,40
10,Sky Temple,40
13,Volskaya Foundry,30


In [100]:
df = player_heroes[['Hero', 'Battletag', 'TotalGames']]
df = df.groupby(['Hero', 'Battletag'])['TotalGames'].max()
x = df.groupby(['Hero', 'Battletag']).cumsum(axis=0).reset_index()

y = pd.crosstab(index=[x.Hero], columns=x.Battletag, values=x.TotalGames, aggfunc='sum', margins=True)
y.head(100)

Battletag,AlixGx#1722,Javedesi#1399,OmarG#1399,RndmEngineer#1403,SmackLoud#1757,cactipus#1355,All
Hero,,,,,,,
Abathur,NaN,NaN,1.0,NaN,4.0,NaN,5
Alexstrasza,1.0,NaN,NaN,1.0,6.0,NaN,8
Ana,NaN,NaN,NaN,NaN,NaN,11.0,11
Anduin,4.0,NaN,NaN,NaN,NaN,7.0,11
Anub'arak,NaN,NaN,NaN,7.0,2.0,NaN,9
...,...,...,...,...,...,...,...
Xul,NaN,10.0,NaN,NaN,12.0,3.0,25
Yrel,1.0,NaN,NaN,2.0,NaN,5.0,8
Zarya,NaN,NaN,7.0,NaN,NaN,NaN,7


In [ ]:
df = player_heroes[['', 'Battletag', 'TotalGames']]
df = df.groupby(['Hero', 'Battletag'])['TotalGames'].max()
x = df.groupby(['Hero', 'Battletag']).cumsum(axis=0).reset_index()

y = pd.crosstab(index=[x.Hero], columns=x.Battletag, values=x.TotalGames, aggfunc='sum', margins=True)
y.head(100)

In [11]:
plt.close("all")

In [15]:
df2 = replays
df_total = df2[['GameMap', 'IsWinner']]
df_total = df_total.reset_index().drop_duplicates()
game_map_total = df_total.groupby(['GameMap']).size().reset_index(name='TotalGames').sort_values(by='TotalGames', ascending=False)
game_map_total.set_index('GameMap', inplace=True)
game_map_total = game_map_total.join(df_total.groupby(['GameMap'])['IsWinner'].sum().reset_index(name='TotalWins').set_index('GameMap'), how='inner')
game_map_total['WinRate'] = (game_map_total['TotalWins'] / game_map_total['TotalGames']) * 100

# game_map_total[['TotalGames', 'WinRate']].plot(kind='barh', title='Map Totals', figsize=(20,10))
# game_map_total[['TotalGames', 'WinRate']].plot(kind='area', title='Map Totals', figsize=(20,10), color=['teal', 'salmon'], subplots=True)

In [16]:
df_total = df2[['Hero', 'IsWinner']]
df_total = df_total.reset_index().drop_duplicates()
hero_total = df_total.groupby(['Hero']).size().reset_index(name='TotalGames').sort_values(by='TotalGames', ascending=False)
hero_total.set_index('Hero', inplace=True)
hero_total = hero_total.join(df_total.groupby(['Hero'])['IsWinner'].sum().reset_index(name='TotalWins').set_index('Hero'), how='inner')
hero_total['WinRate'] = (hero_total['TotalWins'] / hero_total['TotalGames']) * 100

In [18]:
game_map_total.head(100)

,TotalGames,TotalWins,WinRate
GameMap,,,
Infernal Shrines,23425,12242,52.260406
Cursed Hollow,23136,11916,51.504149
Dragon Shire,22889,11879,51.898292
Towers of Doom,22699,11611,51.152033
Tomb of the Spider Queen,22554,11584,51.361178
Battlefield of Eternity,21648,11199,51.732262
Sky Temple,21179,11126,52.533170
Braxis Holdout,21036,10812,51.397604
Garden of Terror,17163,8772,51.109946


In [16]:
# Make traces for graph
# Initialize a figure with ff.create_table(table_data)
fig = ff.create_table(game_map_total.reset_index()[['GameMap', 'TotalGames', 'WinRate']], height_constant=100)
map_index = game_map_total.index.tolist()

print(map_index)

trace1 = go.Bar(x=map_index, y=hero_total['TotalGames'], xaxis='x2', yaxis='y2',
                marker=dict(color='#0099ff'),
                name='Total Games')

trace2 = go.Bar(x=map_index, y=hero_total['WinRate'], xaxis='x2', yaxis='y2',
                marker=dict(color='#404040'),
                name='Win Rate')


# Add trace data to figure
fig.add_traces([trace1, trace2])

# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.yaxis.update({'domain': [0, .45]})
fig.layout.yaxis2.update({'domain': [.6, 1]})

# The graph's yaxis2 MUST BE anchored to the graph's xaxis2 and vice versa
fig.layout.yaxis2.update({'anchor': 'x2'})
fig.layout.xaxis2.update({'anchor': 'y2'})
fig.layout.yaxis2.update({'title': 'Games / Win Rate'})

# Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':75, 'l':50})
fig.layout.update({'title': 'Map Stats'})

# Update the height because adding a graph vertically will interact with
# the plot height calculated for the table
fig.layout.update({'height':800})

# Plot!
fig.show()

['Infernal Shrines', 'Hanamura Temple', 'Tomb of the Spider Queen', 'Garden of Terror', 'Warhead Junction', 'Battlefield of Eternity', 'Dragon Shire', 'Braxis Holdout', 'Sky Temple', 'Towers of Doom', 'Cursed Hollow', 'Alterac Pass', 'Volskaya Foundry', "Blackheart's Bay"]


In [17]:
# Make traces for graph
# Initialize a figure with ff.create_table(table_data)
fig = ff.create_table(hero_total[hero_total['TotalGames'] > 5].reset_index()[['Hero', 'TotalGames', 'WinRate']], height_constant=100)
hero_index = hero_total.index.tolist()

trace1 = go.Scatter(x=hero_index, y=hero_total['TotalGames'], xaxis='x2', yaxis='y2',
                marker=dict(color='#0099ff'),
                name='Total Games')

trace2 = go.Scatter(x=hero_index, y=hero_total['WinRate'], xaxis='x2', yaxis='y2',
                marker=dict(color='#404040'),
                name='Win Rate')


# Add trace data to figure
fig.add_traces([trace1, trace2])

# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.yaxis.update({'domain': [0, .45]})
fig.layout.yaxis2.update({'domain': [.6, 1]})

# The graph's yaxis2 MUST BE anchored to the graph's xaxis2 and vice versa
fig.layout.yaxis2.update({'anchor': 'x2'})
fig.layout.xaxis2.update({'anchor': 'y2'})
fig.layout.yaxis2.update({'title': 'Games / Win Rate'})

# Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':75, 'l':50})
fig.layout.update({'title': 'Hero Stats'})

# Update the height because adding a graph vertically will interact with
# the plot height calculated for the table
fig.layout.update({'height':1000})

# Plot!
fig.show()

In [18]:
fig = None